In [4]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils.utils import pretty_lat, pretty_lon, ProgressStatus, plot_n_steps_of_area_from_nc_file
from tempfile import TemporaryDirectory

In [5]:
from station.data_submission import DataSubmission
from station.station import StationData

from train_station_twin.training_executer import TrainingExecuter
from infilling.evaluation_executer import EvaluationExecuter
from infilling.infilling_writer import InfillingWriter

from era5.era5_for_station import DownloadEra5ForStation, DownloadEra5ForStationGaps
from era5.era5_download_hook import Era5DownloadHook
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


In [6]:
era5_hook = Era5DownloadHook(lat=0, lon=0)
era5_hook.coordinate_limits = {
    "north": 90,
    "south": -90,
    "west": -180,
    "east": 180
}

today = pd.Timestamp.now().date()


era5_hook.download_hours_on_same_day(
    year=today.year,
    month=today.month,
    day=today.day,
    hours=[0,3,6,9,12,15,18,21],
    

In [7]:
barbados_data = DataSubmission("Barbados")
barbados_data.measurement_dir_path = "./measurements/Barbados/"

barbados_station = StationData(
    name=barbados_data.name,
    folder_path=barbados_data.measurement_dir_path
)

Extracting Barbados...


  0%|          | 0/1263 [00:00<?, ?it/s]

 26%|██▌       | 326/1263 [00:30<01:26, 10.83it/s]


KeyboardInterrupt: 

In [ ]:
# start date
start_date = "2020-01-01"
# end date
end_date = "2020-12-31"

# slice the df barbados_station.df
df = barbados_station.df.loc[start_date:end_date]
df

,vis_light,tas,uv_light,ir_light
datetime,,,,
2020-06-23 18:00:00,716.931034,301.148276,246.517241,5135.344828
2020-06-23 19:00:00,512.883333,299.438333,137.083333,2969.266667
2020-06-23 20:00:00,427.733333,299.860000,91.516667,2080.833333
2020-06-23 21:00:00,314.633333,299.188333,30.900000,839.716667
2020-06-23 22:00:00,266.316667,298.666667,5.100000,315.800000
...,...,...,...,...
2020-12-31 19:00:00,511.050000,299.880769,136.116667,2856.783333
2020-12-31 20:00:00,360.931034,298.935714,55.637931,1277.741379
2020-12-31 21:00:00,279.233333,298.241667,11.950000,459.116667


In [ ]:

from utils.utils import plot_measurements_df
plot_measurements_df(barbados_station.df)

**Plot a few random timesteps of the weather station:**

In [ ]:
random_timesteps = marshall_station.df.index

NameError: name 'marshall_station' is not defined

: 

In [ ]:

training = TrainingExecuter(station=marshall_station, ProgressStatus())

output_model_path = training.execute()
marshall_data.add_model(output_model_path)



In [ ]:
barbados_station = StationData("Barbados", "./measurements/Barbados/")
vienna_station = StationData("Vienna", "./measurements/Vienna/")